# Stop Loss / Take Profit

In [1]:
%load_ext autoreload
%autoreload 2
# %matplotlib widget

In [2]:
import sys
from functools import partial
from pathlib import Path
if (path := r'C:\dev\projects\allocation') not in sys.path:
    sys.path.append(path)

    
import numpy as np
import pandas as pd
import ipywidgets as widgets
import plotly.graph_objects as go
from IPython.display import display
from matplotlib import pyplot as plt
from plotly.subplots import make_subplots

from allocation.strategies import TestStrategy, ConstantStopLossStrategy, VolStopLossStrategy
from allocation.app import App

### 0. Data Loading

In [3]:
mapping_ohlc = {
    'UB1 R:05_0_R Comdty': 'EMU_Buxl_Fut',
    'US1 R:05_0_R Comdty': 'US_Bond_30y_Fut',
    'XM1 R:05_0_R Comdty': 'AUD_Bond_10y_Fut',
    'VG1 R:05_0_R Index': 'EMU_Equity_Fut',
    'MES1 R:05_0_R Index': 'EM_Equity_Fut',
    'XP1 R:05_0_R Index': 'AUD_Equity_Fut',
    'PT1 R:05_0_R Index': 'CAD_Equity_Fut',
    'ES1 R:05_0_R Index': 'US_Equity_Fut',
    'Z 1 R:05_0_R Index': 'UK_Equity_F100_Fut',
    'USDCAD Curncy': 'FX_CAD_TR',
    'USDJPY Curncy': 'FX_JPY_TR',
    'EURUSD Curncy': 'FX_EUR_TR',
    'AUDUSD Curncy': 'FX_AUD_TR',
    'USDNOK Curncy': 'FX_NOK_TR',
    'EURSEK Curncy': 'FX_EURSEK_TR',
    'USDMXN Curncy': 'FX_MXN_TR',
}

mapping_hourly = {
    'AUDUSD': 'FX_AUD_TR',
    'USDCAD': 'FX_CAD_TR',
    'EURUSD': 'FX_EUR_TR',
    'USDMXN': 'FX_MXN_TR',
    'USDNOK': 'FX_NOK_TR',
    'EURSEK': 'FX_EURSEK_TR',
    'USDJPY': 'FX_JPY_TR',
    # 'GBPUSD': 'GBPUSD',
    'UB': 'EMU_Buxl_Fut',
    'US': 'US_Bond_30y_Fut',
    'XP': 'AUD_Equity_Fut',
    'ES': 'US_Equity_Fut',
    'VG': 'EMU_Equity_Fut',
    'Z': 'UK_Equity_F100_Fut',
    'PT': 'CAD_Equity_Fut',
}

mapping_sparks = {
   'US_Bond': 'US_Bond_30y_Fut',
    'EURUSD': 'FX_EUR_TR',
    'AUDUSD': 'FX_AUD_TR',
    'Buxl': 'EMU_Buxl_Fut',
    'SP': 'US_Equity_Fut',
    'Eurostoxx': 'EMU_Equity_Fut',
    'EM': 'EM_Equity_Fut',
    'USDJPY': 'FX_JPY_TR',
}

In [4]:
data_path = Path(path) / 'data'
all_ohlc = pd.read_csv(data_path / 'ohlcv.csv', index_col=0, header=[0, 1], parse_dates=True, dayfirst=True).rename(columns=mapping_ohlc)
targets_ohlc = all_ohlc.loc['2005-09-09':, mapping_ohlc.values()]
all_signals = pd.read_csv(data_path / 'signals.csv', index_col=0, parse_dates=True, dayfirst=True)
all_signals_sab = pd.read_csv(data_path / 'signalssab.csv', index_col=0, parse_dates=True, dayfirst=True).rename(columns=mapping_sparks)
targets_hourly = pd.read_csv(data_path / 'hourly.csv', index_col=0, parse_dates=True).rename(columns=mapping_hourly)

In [5]:
app = App(
    signals=all_signals_sab, ohlc_prices=targets_ohlc, 
    hourly_prices= targets_hourly # targets_ohlc.loc[:, (slice(None), 'close')].droplevel(axis=1, level=-1) #
         )
app

In [ ]:
app.freq

In [ ]:
params = {
    'US_Bond_30y_Fut': {'stop_loss': 1.00, 'take_profit': 0, 'vol': 'ewm', 'alpha': 0.1},  
    'EMU_Buxl_Fut': {'stop_loss': 0.8, 'take_profit': 2.1, 'vol': 'std'},
    
    'FX_EURSEK_TR': 'Bad Data',
    'EM_Equity_Fut': 'Bad Data',
    'AUD_Bond_10y_Fut': 'Bad Data',

    'US_Equity_Fut': 'No difference',
    'EMU_Equity_Fut': 'No difference',
    
    'FX_JPY_TR': {'stop_loss': 1.1, 'take_profit': 1.7, 'vol': 'ewm', 'alpha': 0.06},
    'AUD_Equity_Fut': {'stop_loss': 1, 'take_profit': 2, 'vol': 'ewm', 'alpha': 0.05},  # from 2017
    'CAD_Equity_Fut': {'stop_loss': 1.6, 'take_profit': 2, 'vol': 'ewm', 'alpha': 0.1},  
     
    'FX_EUR_TR': {'stop_loss': 1.00, 'take_profit': 0, 'vol': 'ewm', 'alpha': 0.1},    
    'FX_NOK_TR': {'stop_loss': 0.5, 'take_profit': 0, 'vol': 'ewm', 'alpha': 0.1},  # risk profile better

    'UK_Equity_F100_Fut': {'stop_loss': 2.50, 'take_profit': 0.5, 'vol': 'ewm', 'alpha': 0.1},
    'FX_CAD_TR': {'stop_loss': 2.5, 'take_profit': 0.86, 'vol': 'ewm', 'alpha': 0.11}, 
    'FX_AUD_TR': {'stop_loss': 2.5, 'take_profit': 0.9, 'vol': 'std'},  # bit perf, same risk
    'FX_MXN_TR': {'stop_loss': 2, 'take_profit': 0.8, 'vol': 'ewm', 'alpha': 0.1},
    
}

In [ ]:
# N: no difference
# M: data mismatch

params = {
    
    'FX_EURSEK_TR': 'Bad Data',
    'EM_Equity_Fut': 'Bad Data',
    'AUD_Bond_10y_Fut': 'Bad Data',

    'US_Equity_Fut': 'No difference',
    'EMU_Equity_Fut': 'No difference',
    
    'FX_JPY_TR': {'stop_loss': 1.1, 'take_profit': 1.7, 'vol': 'ewm', 'alpha': 0.06},
    'AUD_Equity_Fut': {'stop_loss': 1, 'take_profit': 2, 'vol': 'ewm', 'alpha': 0.05},  # from 2017
    'CAD_Equity_Fut': {'stop_loss': 1.6, 'take_profit': 2, 'vol': 'ewm', 'alpha': 0.1},  
    'US_Bond_30y_Fut': {'stop_loss': 1.00, 'take_profit': 1.5, 'vol': 'ewm', 'alpha': 0.1},   
    'FX_EUR_TR': {'stop_loss': 1.00, 'take_profit': 0, 'vol': 'ewm', 'alpha': 0.1},    
    'FX_NOK_TR': {'stop_loss': 0.5, 'take_profit': 0, 'vol': 'ewm', 'alpha': 0.1},  # risk profile better
    'EMU_Buxl_Fut': {'stop_loss': 0.8, 'take_profit': 2.1, 'vol': 'std'},
    'UK_Equity_F100_Fut': {'stop_loss': 2.50, 'take_profit': 0.5, 'vol': 'ewm', 'alpha': 0.1},
    'FX_CAD_TR': {'stop_loss': 2.5, 'take_profit': 0.86, 'vol': 'ewm', 'alpha': 0.11}, 
    'FX_AUD_TR': {'stop_loss': 2.5, 'take_profit': 0.9, 'vol': 'std'},  # bit perf, same risk
    'FX_MXN_TR': {'stop_loss': 2, 'take_profit': 0.8, 'vol': 'ewm', 'alpha': 0.1},
    
}

In [ ]:
pd.DataFrame(params).T

In [ ]:
target = 'US_Equity_Fut'
freq = 'W-FRI'

In [ ]:
df = targets_ohlc.loc[:, target].dropna()

In [ ]:
df['close'].pct_change().plot()

In [ ]:
target_dates = df.groupby(pd.Grouper(freq=freq)).apply(lambda df: pd.Series([df.loc[:, 'high'].idxmax(), df.loc[:, 'low'].idxmin()], index=['high', 'low']))
high = target_dates['high'].dt.isocalendar().groupby('day')['day'].count()
low = target_dates['low'].dt.isocalendar().groupby('day')['day'].count()

high_ser = target_dates['high'].dt.isocalendar()['day']
low_ser = target_dates['low'].dt.isocalendar()['day']

mat = np.zeros(shape=(5, 5))
for i, j in zip(high_ser, low_ser):
    mat[i-1, j-1] += 1

df_freq = pd.DataFrame(
    mat, 
#         index=pd.MultiIndex.from_product((['high'], ['mon', 'tus', 'wed', 'thu', 'fri']), names=['price', 'day']), 
#         columns=pd.MultiIndex.from_product((['low'], ['MON', 'TUE', 'WED', 'THU', 'FRI']), names=['price', 'day'])
    index=pd.Index(['mon', 'tus', 'wed', 'thu', 'fri'], name='HIGH'),
    columns=pd.Index(['mon', 'tus', 'wed', 'thu', 'fri'], name='LOW')
)

if pct:
    fmt = '.1%'
    df_freq = df_freq / len(target_dates)
else:
    fmt='.3g'

In [ ]:
states = pd.concat([high_ser.rename('high'), low_ser.rename('low')], axis=1)\
           .replace({1: 'MON', 2: 'TUE', 3: 'WED', 4: 'THU', 5: 'FRI'})\
           .apply(tuple, axis=1).astype('category')

In [ ]:
trasition_matrix = np.zeros(shape=(len(states.cat.categories), len(states.cat.categories)))
for j, state in enumerate(states.cat.categories):
    i = states[states.shift()==state]
    ii = i.groupby(i.cat.codes).count() / len(i)
    trasition_matrix[ii.index, j] = ii

In [ ]:
fig, ax = plt.subplots(figsize=(20, 15))
transition_matrix_df = pd.DataFrame(trasition_matrix, index=states.cat.categories, columns=states.cat.categories)
sns.heatmap(transition_matrix_df, ax=ax, annot=True, fmt='.0%', cmap='OrRd')
# ax.invert_yaxis()
ax.xaxis.tick_top()
ax.xaxis.set_tick_params(rotation=90)

In [ ]:
dummies = pd.get_dummies(states)
pd.concat([dummies[dummies.shift().iloc[:, i]==1].mean() for i in range(dummies.shape[1])], axis=1)

In [ ]:

pct = True
import plotly.figure_factory as ff
data = pd.DataFrame(mat).applymap('{:.1%}'.format) if pct else mat
heat = ff.create_annotated_heatmap(z=data.to_numpy(), x=['MON', 'TUE', 'WED', 'THU', 'FRI'], y=['MON', 'TUE', 'WED', 'THU', 'FRI'], colorscale='YlOrRd')
heat.update_xaxes(title_text='HIGHS')
heat.update_yaxes(title_text='LOWS')

fig = make_subplots(rows=2, cols=2)
fig.add_trace(heat.data[0])
fig.update_layout(annotations=heat.layout['annotations'])

In [ ]:
s1 = TestStrategy(
            asset=targets_ohlc.loc[:, (target, 'close')].dropna().resample('W-FRI').last().pct_change(),
            signals=all_signals[target]
        )

s2 = ConstantStopLossStrategy(asset=targets_hourly[target], signals=all_signals[target])

In [ ]:
pnl1= s1.get_pnl()
pnl2 = s2.get_pnl()

In [ ]:
r = targets_ohlc.loc[:, (target, 'close')].pct_change()

In [ ]:
df_m = pd.concat([r, r], axis=1)\
.agg(['mean', 'std',
      lambda x: x.mean() /x.std(), 
      lambda x: x.quantile(0.05), 
      lambda x: x[x<= x.quantile(0.05)].mean()])\
.set_axis(['mean', 'std', 'ir', 'var5', 'es5']).set_axis([0,1], axis=1)

In [ ]:
app = SLTP(all_signals, targets_ohlc, targets_ohlc.loc[:, (slice(None), 'close')].droplevel(axis=1, level=-1))
app

In [ ]:
all_signals.loc[all_signals.first_valid_index():]

In [ ]:
app.date_range_slider._trait_notifiers

In [ ]:
x.index.strftime(f'  {datetime_format}  ')

In [ ]:
app.date_range_slider = 2

In [ ]:
from macro_tilib import get_all_ti, api

In [ ]:
hour_multilier = 24
params = {'fastperiod': 12 * hour_multilier, 'slowperiod': 26 * hour_multilier, 'signalperiod': 9  * hour_multilier}
macd_df = api.MACD(targets_hourly['US_Equity_Fut'].dropna().rename('close').to_frame(), **params).dropna()

In [ ]:
api.MACD

In [ ]:
targets_hourly['US_Equity_Fut'].dropna().loc['2005-11-29']

In [ ]:
macd_df.iloc[500:800,].plot()

In [ ]:
s = all_signals['US_Equity_Fut'].dropna().shift(-1)
s.index = s.index + pd.offsets.Hour(23)

In [ ]:
sm = pd.concat([s, macd_df['macdhist']], join='inner', axis=1)

In [ ]:
ss = ((np.sign(sm).prod(axis=1) > 0) &  (s != 0)) * s
ss.index = ss.index - pd.offsets.Hour(23)
ss = ss.shift()

In [ ]:
r = targets_ohlc.loc[:, ('US_Equity_Fut', 'close')].resample(rule='W-FRI').last().pct_change()

In [ ]:
macd_df.loc['2012-07-20': '2012-08-03']

In [ ]:
(r.reindex(ss.index)[ss == 0] < 0)

In [ ]:
r[ss == 0]

In [ ]:
(r * ss).r.prices(compound=False).plot()
TestStrategy(signals=all_signals['US_Equity_Fut'], asset=targets_ohlc.loc[:, ('US_Equity_Fut', 'close')].resample('W-FRI').last().pct_change()).get_pnl(compound=False).plot()

In [ ]:
all_signals['US_Equity_Fut'].dropna().shift(-1).index 

In [ ]:
macd_df['macdhist'].index 

In [ ]:
pd.offsets.Hour(23)

In [ ]:
ti = get_all_ti(targets_hourly['US_Equity_Fut'].dropna().to_frame())

In [ ]:
pd.DataFrame(api.MACD(targets_hourly['US_Equity_Fut'].dropna()))

In [ ]:
macd = MACD(targets_hourly['US_Equity_Fut'].dropna())

In [ ]:
pd.DataFrame(macd)

In [ ]:
targets_hourly['US_Equity_Fut']